# Advanced v1 решение от команды mipt dudes 

## Библиотеки и таблица с данными

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('data_not_clean.pkl')
df

,query,text,label,clean_text,clean_query,embedding_text,embedding_query
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,1,Зачислен в списки ВМФ СССР 19 августа 1952 год...,Когда был спущен на воду первый миноносец «Спо...,"[-0.0030975677, -0.018142669, -0.0058722952, 0...","[0.00064380514, 0.0074218363, -0.03353223, -0...."
1,Как долго существовало британское телевизионно...,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",1,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",Как долго существовало британское телевизионно...,"[0.0026477594, -0.026646728, 0.0009579654, -0....","[-0.029795218, -0.01173853, -0.00032150946, -0..."
2,Когда родилась Князева Марина Леонидовна?,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,1,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,Когда родилась Князева Марина Леонидовна?,"[-0.036747612, -0.012604811, -0.0109199695, -0...","[-0.036575466, -0.010551005, -0.04117768, -0.0..."
3,Кто был главным художником мира Зен?,"В книге ""Half-Life 2: Raising the Bar"" художни...",1,"В книге ""Half-Life 2: Raising the Bar"" художни...",Кто был главным художником мира Зен?,"[-0.02514373, -0.023727695, -0.04738828, 0.011...","[-0.022960061, -0.013048667, -0.018877652, -0...."
4,Как звали предполагаемого убийцу Джона Кеннеди?,В 1966 году окружной прокурор Нового Орлеана Д...,1,В 1966 году окружной прокурор Нового Орлеана Д...,Как звали предполагаемого убийцу Джона Кеннеди?,"[0.0074619167, -0.024880972, -0.026498705, 0.0...","[-0.0124044325, -0.0020067864, -0.030558525, 0..."
...,...,...,...,...,...,...,...
47016,Кто убил Зорана Джинджича?,Смерть Зорана Джинджича была констатирована в ...,0,Смерть Зорана Джинджича была констатирована в ...,Кто убил Зорана Джинджича?,"[-0.008258977, 0.0024080626, -0.04842605, -0.0...","[-0.052019194, 0.01754323, -0.05360399, -0.031..."
47017,Кто убил Зорана Джинджича?,"Аресты по делу Джинджича, однако, продолжились...",0,"Аресты по делу Джинджича, однако, продолжились...",Кто убил Зорана Джинджича?,"[0.008173082, 0.052790318, -0.05316838, 0.0171...","[-0.052019194, 0.01754323, -0.05360399, -0.031..."
47018,Кто убил Зорана Джинджича?,Ратомир Живкович обвинил в организации убийств...,0,Ратомир Живкович обвинил в организации убийств...,Кто убил Зорана Джинджича?,"[-0.017322723, 0.013281805, -0.039236594, -0.0...","[-0.052019194, 0.01754323, -0.05360399, -0.031..."
47019,Кто убил Зорана Джинджича?,В 12:25 (по свидетельству Биляны Станков и вод...,0,В 12:25 (по свидетельству Биляны Станков и вод...,Кто убил Зорана Джинджича?,"[-0.024740426, -0.006180703, -0.046707638, 0.0...","[-0.052019194, 0.01754323, -0.05360399, -0.031..."


## Метрика 

Интуиция - предполагается что будет дальнейшая обработка результата, так что нам важно хорошо попадать в первые топ N ответов. При этом хочется чтобы все правильные ответы шли выше чем неправильные.  

Пусть n - этого всего ответов на запрос, из них k - правильных.  

Метрика это процент правильных ответов, которые находятся среди первых max(3, k) позиций после ранжирования  

In [5]:
def custom_metric(df_sorted):
    n = len(df_sorted)
    k = df_sorted['label'].sum()
    if k == 0:
        return 1
    top_k = max(3, int(k))
    
    top_k_answers = df_sorted.iloc[:top_k]
    
    correct_in_top_k = top_k_answers['label'].sum()
    
    score = correct_in_top_k / k
    return score


## Ранжирование

Ранжирование основано на идее обучения bert подобных моделей:
  - Один из двух этапов обучения это подача двух предложений, разделённых специальным токеном '\SEP', и вопрос к модели - могут ли эти предложения стоять друг за другом в тексте. На выходе получаем уверенность модели от 0 до 1 в правильном ответе.
  - Казалось бы - то что нам идеально подходит, но есть ньюанс - см результат.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cointegrated/rubert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def rank_answers_by_relevance(df):
    """
    Функция для ранжирования ответов на основе вероятности релевантности вопроса и ответа.
.
    :param df: DataFrame с двумя столбцами 'query' и 'text'
    :return: DataFrame с дополнительным столбцом 'relevance', отсортированный по релевантности
    """
    relevance_scores = []

    for _, row in df.iterrows():
        query = row['query']
        answer = row['text']
        inputs = tokenizer.encode_plus(
            query,
            answer,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=512,
            truncation=True,
            padding='max_length'
        )
        
        with torch.no_grad():
            outputs = model(**inputs)
            relevance_score = torch.softmax(outputs.logits, dim=1)[0][1].item()  # Берем вероятность класса 1 (релевантный)
            relevance_scores.append(relevance_score)
    
    df['relevance'] = relevance_scores
    
    df_sorted = df.sort_values(by='relevance', ascending=False).reset_index(drop=True)
    
    return df_sorted

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
res = []
#i=0
for quey, text in df.groupby('query'):
    #i+=1
    x = rank_answers_by_relevance(text)
    score = custom_metric(x)
    n = len(x)
    k = x['label'].sum()
    res.append([quey,score, n, k])
    #print(quey, 'score =',score,'N =',n, 'K =', k)
    #if i ==6:
    #    break

In [8]:
summ_score = 0
for elem in res:
    summ_score += elem[1]
print(summ_score/len(res) )

0.5075479734157072


## Анализ
Результат плохой, так как модель не дообучается на наших данных, а хорошие эмбеддинги мы не можем использовать здесь, так как на вход подаётся текст. Необходимы модификации, возможно изменение архитектуры. (плавный переход к финальной модели)

без софтмакса

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

def rank_answers(df, model_name='cointegrated/rubert-tiny'):
    # Загрузка модели и токенизатора
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    # Применяем токенизацию с обрезанием длины
    def encode_text(text):
        return tokenizer.encode(text, max_length=512, truncation=True, return_tensors='pt')
    
    # Создаем представления для запросов и ответов
    queries = df['query'].apply(encode_text)
    answers = df['text'].apply(encode_text)
    
    # Рассчитываем сходства (например, косинусное)
    similarities = []
    for query, answer in zip(queries, answers):
        with torch.no_grad():
            query_embedding = model(query).last_hidden_state.mean(dim=1)  # Получаем эмбеддинг для запроса
            answer_embedding = model(answer).last_hidden_state.mean(dim=1)  # Эмбеддинг для ответа
            similarity = torch.cosine_similarity(query_embedding, answer_embedding).item()
            similarities.append(similarity)
    
    # Используем .loc для изменения DataFrame
    df.loc[:, 'similarity'] = similarities
    
    # Сортируем по сходству
    df_sorted = df.sort_values(by='similarity', ascending=False).reset_index(drop=True)
    
    return df_sorted
